In [1]:
from crewai import Crew
from Agents import TripAgents
from Tasks import TripTasks

agents = TripAgents()
tasks = TripTasks()

file_agent = agents.file_reader_agent()
file_reader_task = tasks.file_reader_task(
      file_agent, "InputData.json", "MathOperations.log", "MathOperations.py")
    
from langchain_ollama import ChatOllama
import os
os.environ["OPENAI_API_KEY"] = "NA"

llm = ChatOllama(
    model = "mistral-nemo",
    base_url = "http://localhost:11434")

    
crew = Crew(
            agents=[file_agent,],
            tasks=[file_reader_task,],
            verbose=True,
            manager_llm=llm
        )

result = crew.kickoff()



print("Done")

print(result)

 [2024-09-16 18:05:31][DEBUG]: == Working Agent: File reader
 [2024-09-16 18:05:31][INFO]: == Starting Task: 
You should read following files:
1. inputFile, which is a json file containing input data. Path: InputData.json
2. logFile, which contains logs about code flow execution. Path: MathOperations.log
3. computationCode, which contains python code. This code execution generates logs from input file. Path: MathOperations.py

Return the data in separate sections. For e.g.,
Input file: <input file content>
Log file: <log file content>
Python file: <computation file content>



> Entering new CrewAgentExecutor chain...
Thought: I need to read each file using the "Read a file given file-name" tool.
Action: Read a file given file-name
Action Input: {'name': 'InputData.jsonRead command is executed for file name:InputData.json
[
    {"operation": "add", "a": 5, "b": 3},
    {"operation": "subtract", "a": 10, "b": 4},
    {"operation": "multiply", "a": 7, "b": 6},
    {"operation": "divide",

In [4]:
print(result)

from crewai import Task
from textwrap import dedent
from crewai import Agent, Task, Crew
from langchain_ollama import ChatOllama
from langchain.llms import Ollama
import os
os.environ["OPENAI_API_KEY"] = "NA"

llm = ChatOllama(
    model = "mistral-nemo",
    base_url = "http://localhost:11434")


fileComparisonAgent = Agent(
            role="Compare files",
            goal="Compare contents of file and provide similarity or difference in contents",
            backstory="An expert in understanding JSON files, logs, python code, how execution of python works and ability to correlate all four",
            tools=[
            ],
            verbose=True,
            allow_delegation = False,
            llm=llm
        )

fileComparisonTask = Task(
            description=dedent(f"""
                You will be provided with 3 files contents appended as a single content.
                It will be a combination of JSON file (input file to a code), log file (data logged by the code while trying 
                  to execute code) and code file in python (which when executed on input file, logs the data in log file).
            
                               
            Provided content: {result}


                  You need to compare the code, input file and log file and report any discripencies, if found.
            """),
            agent=fileComparisonAgent,
            expected_output="A detailed analysis about discripency or correlation between logs, input and python code"
)

crew = Crew(
            agents=[fileComparisonAgent,],
            tasks=[fileComparisonTask,],
            verbose=True,
        )

result = crew.kickoff()

print(result)

**

**Input File:**
```json
[
    {"operation": "add", "a": 5, "b": 3},
    {"operation": "subtract", "a": 10, "b": 4},
    {"operation": "multiply", "a": 7, "b": 6},
    {"operation": "divide", "a": 8, "b": 0},
    {"operation": "exponentiate", "a": 2, "b": 3}
]
```

**Log File:**
```
2022-01-01 00:00:00 - Starting operation 'add' with a=5, b=3
2022-01-01 00:00:01 - Operation 'add' completed. Result: 8

2022-01-01 00:00:02 - Starting operation 'subtract' with a=10, b=4
2022-01-01 00:00:03 - Operation 'subtract' completed. Result: 6

2022-01-01 00:00:04 - Starting operation 'multiply' with a=7, b=6
2022-01-01 00:00:05 - Operation 'multiply' completed. Result: 42

2022-01-01 00:00:06 - Starting operation 'divide' with a=8, b=0
2022-01-01 00:00:07 - Operation 'divide' completed. Result: Infinity

2022-01-01 00:00:08 - Starting operation 'exponentiate' with a=2, b=3
2022-01-01 00:00:09 - Operation 'exponentiate' completed. Result: 8
```

**Python File:**
```python
import json
import loggi

2024-09-16 18:17:15,563 - 47984 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-09-16 18:17:15][DEBUG]: == Working Agent: Compare files
 [2024-09-16 18:17:15][INFO]: == Starting Task: 
                You will be provided with 3 files contents appended as a single content.
                It will be a combination of JSON file (input file to a code), log file (data logged by the code while trying 
                  to execute code) and code file in python (which when executed on input file, logs the data in log file).


            Provided content: **

**Input File:**
```json
[
    {"operation": "add", "a": 5, "b": 3},
    {"operation": "subtract", "a": 10, "b": 4},
    {"operation": "multiply", "a": 7, "b": 6},
    {"operation": "divide", "a": 8, "b": 0},
    {"operation": "exponentiate", "a": 2, "b": 3}
]
```

**Log File:**
```
2022-01-01 00:00:00 - Starting operation 'add' with a=5, b=3
2022-01-01 00:00:01 - Operation 'add' completed. Result: 8

2022-01-01 00:00:02 - Starting operation 'subtract' with a=10, b=4
2022-01-01 00:00:03 - Operation 'subtract' 